# ElasticSearch Data Loading

In [ ]:
import json, time, os
from elasticsearch import Elasticsearch

### ES Index
Build ES data index from JSONS

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [ ]:
settings = {
    
    "test_text":{
        "mappings":{
            "dynamic":"strict",
            "properties":{
                "author":{"type":"text"},
                "content":{"type":"text"},
                "date":{"type":"date","format":"yyyy-MM-dd"},
                "id":{"type":"text"},
                "publication":{"type":"text"},
                "title":{"type":"text"}
                }
            }
        }
    }

In [ ]:
index_name = "corpus2" # Name of the index

In [ ]:
es.create(index=index_name, id=1, body=settings)

In [ ]:
os.chdir("C:/Users/hkhan/OneDrive - LMI/Desktop/Workspace/epic/all-the-news-json")

In [ ]:
## Running through nested json files and indexing each on elastic

for i1, file in enumerate(os.listdir()):
    
    i1+=1
    
        with open(file) as json_file:
           
            f = json.load(json_file)
            
            for i2, j in enumerate(f):
                es.index(index=index_name, id=int(f"{i1}{i2}"), body=j)

### ES Snapshot
Create and restore ES snapshots

In [ ]:
## Close ES index

es.indices.close(index='corpus1')

In [ ]:
## Create repository

repository_body = {
    "type": "fs",
    "settings": {
            "location": "C:/Users/hkhan/OneDrive - LMI/Desktop/Workspace/es-backup"
    }
}

es.snapshot.create_repository(repository='corpus1_repository', body=snapshot_body)

In [ ]:
## Create snapshot in repository

snapshot_body = {
  "indices": "corpus1",
  "ignore_unavailable": True,
  "include_global_state": False,
  "metadata": {
    "taken_by": "Hasan",
    "taken_because": "Creating snapshot for git repo"
  }
}

es.snapshot.create(repository='corpus1_repository', snapshot='corpus1_snapshot', body=snapshot_body)

In [ ]:
## Retrive snapshot


restore_body = {
  "indices": "corpus1",
  "ignore_unavailable": True,
  "include_global_state": True,
  "rename_pattern": "index_(.+)",
  "rename_replacement": "restored_index_$1"
}

es.snapshot.restore(repository='corpus1_repository', snapshot='corpus1_snapshot', body=restore_body)
